<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection

# Modell
from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

In [5]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [7]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag

from oppdeling_trening_testsett import umarkert_andel_enzym
from oppdeling_trening_testsett import umarkert_andel_dag
from oppdeling_trening_testsett import enzymfordeling

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_semiveiledet import BHD
from moduler_optimalisering_evaluering_semiveiledet import SelfTrainingRandomForestRegressor

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_selvtrent_rfr_dag

from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_1
from moduler_optimalisering_evaluering_semiveiledet import semi_modell_evaluering_2

from moduler_optimalisering_evaluering_semiveiledet import bhd_andel_maks
from moduler_optimalisering_evaluering_semiveiledet import andel_umarkert

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28976 entries, 2022-10-31 17:37:00 to 2023-06-14 01:02:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  28976 non-null  int64  
 1   EnzymeType_A2  28976 non-null  int64  
 2   EnzymeType_B   28976 non-null  int64  
 3   EnzymeType_C   28976 non-null  int64  
 4   EnzymeType_D   28976 non-null  int64  
 5   EnzymeType_E   28976 non-null  int64  
 6   RawMatPercent  28976 non-null  float64
 7   NIRfat         28976 non-null  float64
 8   NIRash         28976 non-null  float64
 9   NIRwater       28976 non-null  float64
 10  TT08           28976 non-null  float64
 11  TT20           28976 non-null  float64
 12  TT12           28976 non-null  float64
 13  Mw             455 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [ ]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18510 entries, 2022-11-02 00:00:00 to 2023-06-14 01:02:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18510 non-null  int64  
 1   EnzymeType_A2  18510 non-null  int64  
 2   EnzymeType_B   18510 non-null  int64  
 3   EnzymeType_C   18510 non-null  int64  
 4   EnzymeType_D   18510 non-null  int64  
 5   EnzymeType_E   18510 non-null  int64  
 6   RawMatPercent  18510 non-null  float64
 7   NIRfat         18510 non-null  float64
 8   NIRash         18510 non-null  float64
 9   NIRwater       18510 non-null  float64
 10  TT08           18510 non-null  float64
 11  TT20           18510 non-null  float64
 12  TT12           18510 non-null  float64
 13  Mw             123 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [ ]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10466 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10466 non-null  int64  
 1   EnzymeType_A2  10466 non-null  int64  
 2   EnzymeType_B   10466 non-null  int64  
 3   EnzymeType_C   10466 non-null  int64  
 4   EnzymeType_D   10466 non-null  int64  
 5   EnzymeType_E   10466 non-null  int64  
 6   RawMatPercent  10466 non-null  float64
 7   NIRfat         10466 non-null  float64
 8   NIRash         10466 non-null  float64
 9   NIRwater       10466 non-null  float64
 10  TT08           10466 non-null  float64
 11  TT20           10466 non-null  float64
 12  TT12           10466 non-null  float64
 13  Mw             332 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [ ]:
# Lagring av resultater fra testing og trening
dict_res_1_hel_test = {"modell": [],
                       "test_andel": [],
                       "trening_andel": [],
                       "trening_markert_antall": [],
                       "umarkert_antall": [],
                       "umarkert_benyttet": [],
                       "tid_trening": [],
                       "parametere": [],
                       "RMSE_test": [],
                       "R2_test":[],
                       "MAE_test":[],
                       "MAPE_test":[],
                       "n_runder": [],
                       "n_splitt": [],
                       "tid_optimalisering": []}

dict_res_1_design_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test = copy.deepcopy(dict_res_1_hel_test)

# Uten NIR
dict_res_1_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_1_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)

dict_res_1_hel_trening = {"modell": [],
                          "test_andel": [],
                          "trening_andel": [],
                          "umarkert_antall": [],
                          "umarkert_benyttet": [],
                          "parametere": [],
                          "n_splitt": [],
                          "RMSE_val_snitt": [],
                          "RMSE_val_std": [],
                          "R2_val_snitt":[],
                          "R2_val_std":[],
                          "MAE_val_snitt":[],
                          "MAE_val_std":[],
                          "MAPE_val_snitt":[],
                          "MAPE_val_std":[],
                          "RMSE_trening_snitt": [],
                          "RMSE_trening_std": [],
                          "R2_trening_snitt":[],
                          "R2_trening_std":[],
                          "MAE_trening_snitt":[],
                          "MAE_trening_std":[],
                          "MAPE_trening_snitt":[],
                          "MAPE_trening_std":[],
                          }

dict_res_1_design_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening = copy.deepcopy(dict_res_1_hel_trening)

# Uten NIR
dict_res_1_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_1_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [16]:
# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)


print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
A2               125  34.34
E                 60  16.48
B                 56  15.38
C                 55  15.11
D                 50  13.74
A1                18   4.95
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                31  34.07
E                 15  16.48
B                 14  15.38
C                 14  15.38
D                 13  14.29
A1                 4   4.40
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
A2             17934  62.88
A1              2823   9.90
D               2193   7.69
B               2043   7.16
C               1834   6.43
E               1694   5.94


In [ ]:
# Kryssvalidering
splitt = 10
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                         trening_dict_res=dict_res_1_hel_trening,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         gjentagelser=gjentagelser,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

## Alternativ 1: Forskjellige markerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, enzymtyper_treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    enzymtyper_treningssett_umarkert = enzymtyper_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                            trening_dict_res=dict_res_1_hel_trening,
                            treningssett_markert=treningssett_markert,
                            enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            splitt=splitt,
                            gjentagelser=gjentagelser,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Trening umarkert
    Frekvens  Andel
A2     17987  62.92
A1      2823   9.87
D       2193   7.67
B       2043   7.15
C       1846   6.46
E       1697   5.94


## Alternativ 1: Forskjellige umarkerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandel
trenings_andel = 1.0

# Andeler markert fra umarkert
andel_av_markert = [0.25, 0.5, 1.0, 2.0]

# Markert sett
treningssett_markert = trening_markert
enzymtyper_treningssett_markert = enzymtyper_markert

# Går gjennom andeler av satt treningssett
for andel in andel_av_markert:

  umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

  print(umarkert_andel)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _= train_test_split(trening_umarkert,
                                                  enzymtyper_umarkert,
                                                  test_size=ubenyttet_andel,
                                                  stratify=enzymtyper_umarkert,
                                                  random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                           axis=0)
  treningssett = treningssett.sort_index(ascending=True)

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                          trening_dict_res=dict_res_1_hel_trening,
                          treningssett_markert=treningssett_markert,
                          enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          gjentagelser=gjentagelser,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Trening umarkert
    Frekvens  Andel
A2     17987  62.92
A1      2823   9.87
D       2193   7.67
B       2043   7.15
C       1846   6.46
E       1697   5.94


Lagring av resultater

In [ ]:
mw_res_1_hel_trening_semi = pd.DataFrame(dict_res_1_hel_trening)
mw_res_1_hel_test_semi = pd.DataFrame(dict_res_1_hel_test)

mw_res_1_hel_trening_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_hel_trening_semi.csv')
mw_res_1_hel_test_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_hel_test_semi.csv')

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [17]:
# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
A2               131  35.89
E                 60  16.44
B                 54  14.79
C                 54  14.79
D                 50  13.70
A1                16   4.38
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
A2             17934  62.88
A1              2823   9.90
D               2193   7.69
B               2043   7.16
C               1834   6.43
E               1694   5.94


In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                         trening_dict_res=dict_res_2_hel_trening,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed):

## Alternativ 2: Forskjellige markerte treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                          trening_dict_res=dict_res_2_hel_trening,
                          rå_data=rå_data,
                          treningssett_markert=treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Enzymtyper Fordeling: Test
A2    26
B     16
C     15
E     15
D     13
A1     6
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


## Alternativ 2: Forskjellige umarkerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandel
trenings_andel = 1.0

# Andeler markert fra umarkert
andel_av_markert = [0.25, 0.5, 1.0, 2.0]

# Markert sett
treningssett_markert = trening_markert

# Går gjennom andeler av satt treningssett
for andel in andel_av_markert:

  umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

  print(umarkert_andel)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                           axis=0)
  treningssett = treningssett.sort_index(ascending=True)

  # Finner antall av enzymtype med minst antall
  enzym_markert = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                          trening_dict_res=dict_res_2_hel_trening,
                          rå_data=rå_data,
                          treningssett_markert=treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Lagring av resultater

In [ ]:
mw_res_2_hel_trening_semi = pd.DataFrame(dict_res_2_hel_trening)
mw_res_2_hel_test_semi = pd.DataFrame(dict_res_2_hel_test)

mw_res_2_hel_trening_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_hel_trening_semi.csv')
mw_res_2_hel_test_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_hel_test_semi.csv')

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [18]:
# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)


print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
E                 60  22.64
C                 53  20.00
B                 52  19.62
D                 48  18.11
A2                38  14.34
A1                14   5.28
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 15  22.39
B                 13  19.40
C                 13  19.40
D                 12  17.91
A2                10  14.93
A1                 4   5.97
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
D               1926  19.01
A1              1826  18.02
B               1728  17.05
C               1699  16.77
E               1631  16.09
A2              1324  13.06


In [ ]:
# Kryssvalidering
splitt = 10
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                         trening_dict_res=dict_res_1_design_trening,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         gjentagelser=gjentagelser,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

## Alternativ 1: Forskjellige markerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, enzymtyper_treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    enzymtyper_treningssett_umarkert = enzymtyper_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                            trening_dict_res=dict_res_1_design_trening,
                            treningssett_markert=treningssett_markert,
                            enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                            treningssett_umarkert=treningssett_umarkert,
                            enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                            treningssett=treningssett,
                            testsett=testsett,
                            splitt=splitt,
                            gjentagelser=gjentagelser,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Trening umarkert
    Frekvens  Andel
A2     17987  62.92
A1      2823   9.87
D       2193   7.67
B       2043   7.15
C       1846   6.46
E       1697   5.94


## Alternativ 1: Forskjellige umarkerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandel
trenings_andel = 1.0

# Andeler markert fra umarkert
andel_av_markert = [0.25, 0.5, 1.0, 2.0]

# Markert sett
treningssett_markert = trening_markert
enzymtyper_treningssett_markert = enzymtyper_markert

# Går gjennom andeler av satt treningssett
for andel in andel_av_markert:

  umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

  print(umarkert_andel)

  # Filtrer umarkert sett
  ubenyttet_andel = (1.0 - umarkert_andel)
  treningssett_umarkert, _, enzymtyper_treningssett_umarkert, _= train_test_split(trening_umarkert,
                                                  enzymtyper_umarkert,
                                                  test_size=ubenyttet_andel,
                                                  stratify=enzymtyper_umarkert,
                                                  random_state=random_seed)

  # Treningssett
  treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                           axis=0)
  treningssett = treningssett.sort_index(ascending=True)

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                          trening_dict_res=dict_res_1_design_trening,
                          treningssett_markert=treningssett_markert,
                          enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          enzymtyper_treningssett_umarkert=enzymtyper_treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          gjentagelser=gjentagelser,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Trening umarkert
    Frekvens  Andel
A2     17987  62.92
A1      2823   9.87
D       2193   7.67
B       2043   7.15
C       1846   6.46
E       1697   5.94


Lagring av resultater

In [ ]:
mw_res_1_design_trening_semi = pd.DataFrame(dict_res_1_design_trening)
mw_res_1_design_test_semi = pd.DataFrame(dict_res_1_design_test)

mw_res_1_design_trening_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_design_trening_semi.csv')
mw_res_1_design_test_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_design_test_semi.csv')

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [19]:
# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymfordeling(rå_data, trening_markert))
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymfordeling(rå_data, trening_umarkert))

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
E                 60  22.56
C                 53  19.92
B                 52  19.55
D                 48  18.05
A2                39  14.66
A1                14   5.26
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 15  22.73
B                 13  19.70
C                 13  19.70
D                 12  18.18
A2                 9  13.64
A1                 4   6.06
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
D               1926  19.01
A1              1826  18.02
B               1728  17.05
C               1699  16.77
E               1631  16.09
A2              1324  13.06


In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                         trening_dict_res=dict_res_2_design_trening,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed):

## Alternativ 2: Forskjellige markerte treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                          trening_dict_res=dict_res_2_design_trening,
                          rå_data=rå_data,
                          treningssett_markert=treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Enzymtyper Fordeling: Test
A2    26
B     16
C     15
E     15
D     13
A1     6
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


## Alternativ 2: Forskjellige umarkerte treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandel
trenings_andel = 1.0

# Andeler markert fra umarkert
andel_av_markert = [0.25, 0.5, 1.0, 2.0]

# Markert sett
treningssett_markert = trening_markert

# Går gjennom andeler av satt treningssett
for andel in andel_av_markert:

  umarkert_andel = andel_umarkert(andel, treningssett_markert, trening_umarkert)

  print(umarkert_andel)

  # Filtrer umarkert sett
  treningssett_umarkert, _ = umarkert_andel_dag(trening_umarkert,
                                                umarkert_andel)

  # Treningssett
  treningssett = pd.concat([treningssett_markert, treningssett_umarkert],
                           axis=0)
  treningssett = treningssett.sort_index(ascending=True)

  # Finner antall av enzymtype med minst antall
  enzym_markert = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  semi_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                          trening_dict_res=dict_res_2_design_trening,
                          rå_data=rå_data,
                          treningssett_markert=treningssett_markert,
                          treningssett_umarkert=treningssett_umarkert,
                          treningssett=treningssett,
                          testsett=testsett,
                          splitt=splitt,
                          n_runder=n_runder,
                          test_andel=test_andel,
                          trenings_andel=trenings_andel,
                          random_seed=random_seed)

Lagring av resultater

In [ ]:
mw_res_2_design_trening_semi = pd.DataFrame(dict_res_2_design_trening)
mw_res_2_design_test_semi = pd.DataFrame(dict_res_2_design_test)

mw_res_2_design_trening_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_design_trening_semi.csv')
mw_res_2_design_test_semi.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_design_test_semi.csv')

# NIR målinger ekskludert

In [15]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
A2               125  34.34
E                 60  16.48
B                 56  15.38
C                 55  15.11
D                 50  13.74
A1                18   4.95
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
A2             17934  62.88
A1              2823   9.90
D               2193   7.69
B               2043   7.16
C               1834   6.43
E               1694   5.94
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                31  34.07
E                 15  16.48
B                 14  15.38
C                 14  15.38
D                 13  14.29
A1                 4   4.40


In [ ]:
# Kryssvalidering
splitt = 10
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(test_dict_res=dict_res_1_hel_test_NIR,
                         trening_dict_res=dict_res_1_hel_trening_NIR,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         gjentagelser=gjentagelser,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
mw_res_1_hel_trening_semi_NIR = pd.DataFrame(dict_res_1_hel_trening_NIR)
mw_res_1_hel_test_semi_NIR = pd.DataFrame(dict_res_1_hel_test_NIR)

mw_res_1_hel_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_hel_trening_semi_NIR.csv')
mw_res_1_hel_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_hel_test_semi_NIR.csv')

## Alternativ 2: Manuell fordeling

In [20]:
# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

Enzymtyper Fordeling: Trening markert
            Frekvens  Andel
EnzymeCode                 
A2               131  35.89
E                 60  16.44
B                 54  14.79
C                 54  14.79
D                 50  13.70
A1                16   4.38
Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67
Enzymtyper Fordeling: Trening umarkert
            Frekvens  Andel
EnzymeCode                 
A2             17934  62.88
A1              2823   9.90
D               2193   7.69
B               2043   7.16
C               1834   6.43
E               1694   5.94


In [ ]:
# Antall runder til optimalisering
n_runder = 10

# Kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(test_dict_res=dict_res_2_hel_test_NIR,
                         trening_dict_res=dict_res_2_hel_trening_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
mw_res_2_hel_trening_semi_NIR = pd.DataFrame(dict_res_2_hel_trening_NIR)
mw_res_2_hel_test_semi_NIR = pd.DataFrame(dict_res_2_hel_test_NIR)

mw_res_2_hel_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_hel_trening_semi_NIR.csv')
mw_res_2_hel_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_hel_test_semi_NIR.csv')

## Designproduksjon

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [22]:
# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Kryssvalidering
splitt = 10
gjentagelser = 10


# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_1(test_dict_res=dict_res_1_design_test_NIR,
                         trening_dict_res=dict_res_1_design_trening_NIR,
                         treningssett_markert=trening_markert,
                         enzymtyper_treningssett_markert=enzymtyper_markert,
                         treningssett_umarkert=trening_umarkert,
                         enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         gjentagelser=gjentagelser,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

Beste modell

Lagring av resultater

In [ ]:
mw_res_1_design_trening_semi_NIR = pd.DataFrame(dict_res_1_design_trening_NIR)
mw_res_1_design_test_semi_NIR = pd.DataFrame(dict_res_1_design_test_NIR)

mw_res_1_design_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_design_trening_semi_NIR.csv')
mw_res_1_design_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_1_design_test_semi_NIR.csv')

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [23]:
# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaulering av Coreg, Selv-RFR og BHD
semi_modell_evaluering_2(test_dict_res=dict_res_2_design_test_NIR,
                         trening_dict_res=dict_res_2_design_trening_NIR,
                         rå_data=rå_data,
                         treningssett_markert=trening_markert,
                         treningssett_umarkert=trening_umarkert,
                         treningssett=trening,
                         testsett=testsett,
                         splitt=splitt,
                         n_runder=n_runder,
                         test_andel=test_andel,
                         trenings_andel=trenings_andel,
                         random_seed=random_seed)

[I 2024-04-15 19:06:18,045] A new study created in memory with name: no-name-6ac8b6ed-7bbf-4fb7-a8a8-b178b56025e4
[I 2024-04-15 19:09:13,410] Trial 0 finished with value: 736.4857465024405 and parameters: {'k1': 8, 'k2': 3, 'p1': 4, 'p2': 2, 'max_iters': 41, 'pool_size': 147}. Best is trial 0 with value: 736.4857465024405.
[I 2024-04-15 19:10:40,648] A new study created in memory with name: no-name-acc00074-debf-4d41-9dcc-c8c97c6d4d2a
[I 2024-04-15 19:10:54,833] Trial 0 finished with value: 896.5680238588425 and parameters: {'n_estimators': 418, 'criterion': 'absolute_error', 'max_depth': 26, 'min_samples_leaf': 3, 'max_features': 0.700277228925558, 'maks_iterasjoner': 3, 'std_terskel': 0.024469804644067825}. Best is trial 0 with value: 896.5680238588425.
[I 2024-04-15 19:11:08,294] A new study created in memory with name: no-name-62787132-0f91-4b8d-83e1-f13f7a23724e
[I 2024-04-15 19:12:01,560] Trial 0 finished with value: 2341.8098801171654 and parameters: {'k': 5035, 'alpha': 0.43462

Beste modell

Lagring av resultater

In [ ]:
mw_res_2_design_trening_semi_NIR = pd.DataFrame(dict_res_2_design_trening_NIR)
mw_res_2_design_test_semi_NIR = pd.DataFrame(dict_res_2_design_test_NIR)

mw_res_2_design_trening_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_design_trening_semi_NIR.csv')
mw_res_2_design_test_semi_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_res_2_design_test_semi_NIR.csv')